LRU Cache.  
Note that we basically use double linked list and dictionary to implement one version of OrderedDict. In the LFC later, we directly use OrderedDict

In [3]:
class Node():
    def __init__(self, k,v):
        self.key = k
        self.val = v
        self.next = None
        self.prev = None
    
class LRUCache:    
    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.capacity = capacity
        self.head = None
        self.tail = None        
        self.n = 0 
        self.nodeDict = dict() # key value pair where value would be node 
    def moveToHead(self, cur_node):
        if cur_node != self.head:
            # we only need to do anything when cur_node is not already the head

            
            # first of all connect prev and next of cur_node
            cur_node.prev.next = cur_node.next
            if cur_node.next is not None:
                cur_node.next.prev = cur_node.prev
            else:
                # cur_node must be self.tail, after move cur_node to the head, we need to update self.tail
                self.tail = cur_node.prev
            # next
            temp = self.head
            cur_node.next, self.head.prev, self.head =self.head, cur_node, cur_node
        
    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        if key not in self.nodeDict:
            return -1
        cur_node = self.nodeDict[key]
        
        # next we move cur_node to the head while keeping everything else in order
        self.moveToHead(cur_node)
        
        
        # finally return the value
        return cur_node.val
    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        
        if key not in self.nodeDict:
            self.n += 1
            # create a new node
            new_node = Node(key, value)
            # attach to the head
            if self.head is not None:
                new_node.next, self.head.prev, self.head = self.head, new_node, new_node
            else:
                # we are inserting the first element
                self.head = self.tail = new_node                    

            # save it to dict
            self.nodeDict[key] = new_node

            # check whether we need to delete the least recently used
            if self.n > self.capacity:
                del self.nodeDict[self.tail.key]
                self.tail = self.tail.prev
                if self.tail is not None:
                    self.tail.next = None

                self.n -= 1
        else:
            # key already exists 
            cur_node = self.nodeDict[key]
            # modify value 
            cur_node.val = value
            
            # move to the front
            self.moveToHead(cur_node)
            
        
# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

A ordered dict implementation 

In [4]:
from collections import OrderedDict
class LRUCache:
    
    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.capacity = capacity  
         
        self.nodeDict = OrderedDict() # store key value pair while keeping r
        
    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        if key not in self.nodeDict:
            return -1
        cur_node_val = self.nodeDict[key]
        del self.nodeDict[key]        
        self.nodeDict[key] = cur_node_val                
        # finally return the value
        return cur_node_val
    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        `
        if key not in self.nodeDict:            
            self.nodeDict[key] = value

            # check whether we need to delete the least recently used
            if len(self.nodeDict) > self.capacity:
                self.nodeDict.popitem(last = False)                
        else:
            # key already exists 
            del self.nodeDict[key]
            # modify value 
            self.nodeDict[key] = value 
            
        
# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

SyntaxError: invalid syntax (<ipython-input-4-738d47a389f9>, line 30)

LFU Cache

In [ ]:
from collections import OrderedDict
from collections import defaultdict
class LFUCache:

    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.countDict = dict() # value in this dict is a [value, count]
        self.countLRC = defaultdict(OrderedDict) # key is count and value is an OrderedDict that keeps track of the least recently used item
        self.capacity = capacity 
        self.n = 0
        self.minCount = 0
    
    def move_to_end(self, key, newValue = None):
        old_count = self.countDict[key]        
        self.countDict[key] = old_count + 1
        if newValue:
            value = newValue
        else:
            value = self.countLRC[old_count][key]
        del self.countLRC[old_count][key]
        
        if self.minCount == old_count and len(self.countLRC[old_count])==0:
            self.minCount += 1

        self.countLRC[old_count+1][key] = value # LRC is taken care by orderedDict
        
        return value
    
    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        if key not in self.countDict:
            return -1
        
        value = self.move_to_end(key)

        return value

    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        if key in self.countDict:
            _ = self.move_to_end(key, value)
        else:
            if self.capacity > 0:
                # check whether we need to delete the least frequently used item
                if self.n == self.capacity:
                    to_del_key, to_del_value = self.countLRC[self.minCount].popitem(last = False)
                    self.countDict.pop(to_del_key) 
                else:
                    self.n += 1

                # since we just inserted a new item, the minCount must be 1!
                self.minCount = 1                        
                self.countDict[key] = 1
                self.countLRC[1][key] = value

# Your LFUCache object will be instantiated and called as such:
# obj = LFUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)